In [1]:
import sys
sys.path.append("..")

from methods.dcbo import DynCausalBayesOpt
from utils.sequential_sampling import draw_samples_from_sem_dev
from causal_graph.example_dyn_graphs import three_step_stat
from sem.stationary import StationaryModel_dev
from collections import OrderedDict
from utils.costs import equal_cost
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from collections import OrderedDict
tfd = tfp.distributions

In [2]:
sem_model = StationaryModel_dev()
num_samples = 20
temporal_index = 2
full_samples = OrderedDict([(key, []) for key in sem_model.static().keys()])
epsilon = OrderedDict(
    [
        (key, tfd.Normal(0.0, 0.3).sample((num_samples, temporal_index+1)))
        for key in full_samples.keys()
    ]
)

epsilon_x1 = tf.linspace(-5.0, 5.0, num_samples)[:, tf.newaxis]
epsilon_x23 = tfd.Normal(0.0, 0.3).sample((num_samples, 2))
epsilon["X"] = tf.concat([epsilon_x1, epsilon_x23], axis=1)

In [3]:
dyn_graph = three_step_stat()

# D_obs = draw_samples_from_sem_dev(sem_model, 20, 2, seed=1111)
D_obs = draw_samples_from_sem_dev(sem_model, num_samples, temporal_index, epsilon=epsilon)

intervention_ini = {
    "X": [-3.6],
    "Z": [None],
    "Y": [None],
}
D_intervene_ini_x = draw_samples_from_sem_dev(
    sem_model, 1, 0, intervention=intervention_ini, epsilon=0.0
)
intervention_ini = {
    "X": [None],
    "Z": [-4.0],
    "Y": [None],
}
D_intervene_ini_z = draw_samples_from_sem_dev(
    sem_model, 1, 0, intervention=intervention_ini, epsilon=0.0
)
D_intervene_ini = OrderedDict(
    [(("X",), D_intervene_ini_x), (("Z",), D_intervene_ini_z)]
)
intervention_domain = OrderedDict([("X", [-5.0, 5.0]), ("Z", [-5.0, 20.0])])
num_trials = 20
task = "min"
cost_fn = equal_cost
num_anchor_points = 100
num_monte_carlo = 20
jitter = 1e-6
dcbo = DynCausalBayesOpt(
    dyn_graph,
    sem_model,
    D_obs,
    D_intervene_ini,
    intervention_domain,
    num_trials,
    task,
    cost_fn,
    num_anchor_points,
    num_monte_carlo,
    jitter,
)

In [4]:
opt_history = dcbo.run()

Early stopping at step 10669
Early stopping at step 1674
Early stopping at step 1674
Dynamic causal Bayesian optimization at time step 0 is started.
Early stopping at step 2635
Early stopping at step 1825
es ('X',)
candidate_points tf.Tensor(
[[ 2.9636517 ]
 [ 1.035429  ]
 [-1.6976178 ]
 [-4.18406   ]
 [-0.20741367]
 [ 3.4694958 ]
 [ 4.400135  ]
 [-2.0312405 ]
 [-1.4317667 ]
 [-2.2459126 ]
 [-1.004101  ]
 [ 4.447404  ]
 [-4.051963  ]
 [ 2.766614  ]
 [ 0.36584854]
 [ 0.00574827]
 [-2.3350525 ]
 [-3.2022333 ]
 [-1.1351109 ]
 [-1.3672411 ]
 [ 3.2000494 ]
 [ 4.9543715 ]
 [ 2.981851  ]
 [-3.1879592 ]
 [-4.9932137 ]
 [ 4.6615715 ]
 [-2.324605  ]
 [ 3.8370314 ]
 [-4.17169   ]
 [-1.6383553 ]
 [ 1.2508211 ]
 [-2.8077745 ]
 [-4.306817  ]
 [ 2.0906806 ]
 [ 4.3471184 ]
 [-1.6323853 ]
 [-4.3320093 ]
 [-3.0548143 ]
 [-4.610633  ]
 [-1.6150177 ]
 [-4.9053392 ]
 [-4.8911915 ]
 [ 2.7155948 ]
 [-4.4428253 ]
 [ 0.23250675]
 [ 2.6713285 ]
 [ 3.2818403 ]
 [-4.616785  ]
 [-4.383955  ]
 [ 4.8791494 ]
 [-3.74

KeyboardInterrupt: 

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for i in range(3):
    axs[i].plot(opt_history[i])
plt.show()